# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.20.0'

## Part 1 - Data Preprocessing

## Preprocessing and spliting of dataset

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = r"Dataset/Chest_XRay_MSI/./"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    
)

training_set = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),  
    
    class_mode='binary',
    subset='training'
)

validation_set = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
   
    class_mode='binary',
    subset='validation'
)




Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [4]:
print(training_set.class_indices)

{'Malignant': 0, 'Normal': 1}


In [5]:
import numpy as np

print("Class indices:", training_set.class_indices)

class_counts = np.bincount(training_set.classes)
class_labels = list(training_set.class_indices.keys())

for label, count in zip(class_labels, class_counts):
    print(f"{label}: {count}")



Class indices: {'Malignant': 0, 'Normal': 1}
Malignant: 400
Normal: 400


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding='same',
    activation='relu',
    input_shape=(128, 128, 3)
))


C:\Users\HT\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(pool_size=2)


<MaxPooling2D name=max_pooling2d_1, built=True>

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=8, activation='relu'))


### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x=training_set, validation_data=validation_set, epochs=10)


C:\Users\HT\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 189ms/step - accuracy: 0.5000 - loss: 0.9932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 237ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 285ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 352ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 259ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 281ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 254ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.

# Training the VGG16 model

In [15]:
import tensorflow as tf

base_model = tf.keras.applications.VGG16(weights='imagenet',
                                         include_top=False,
                                         input_shape=(128,128,3))
base_model.trainable = False

cnn = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

cnn.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
            loss='binary_crossentropy',
            metrics=['accuracy'])

history = cnn.fit(
    training_set,
    validation_data=validation_set,
    epochs=10
)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 102s 4s/step - accuracy: 0.4837 - loss: 0.7947 - val_accuracy: 0.5400 - val_loss: 0.6896
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.5525 - loss: 0.7133 - val_accuracy: 0.5150 - val_loss: 0.6916
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5575 - loss: 0.6910 - val_accuracy: 0.5500 - val_loss: 0.7000
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5962 - loss: 0.6633 - val_accuracy: 0.5850 - val_loss: 0.6943
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.5813 - loss: 0.6751 - val_accuracy: 0.4550 - val_loss: 0.7317
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.6338 - loss: 0.6410 - val_accuracy: 0.5600 - val_loss: 0.7181
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.6263 - loss: 0.6457 - val_accuracy: 0.5450 - val_loss: 0.7098
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.6100 - loss: 0.6398 - val_accuracy: 0.5450 - val_loss